In [13]:
from aiohttp import ClientSession
import api_setup
import base64
import requests
import asyncio
import json

env_vars = api_setup.parse_api_kvs("./../api-keys")

In [14]:
def get_header_with_token(client_id: str, client_secret: str):
    creds = f"{env_vars['client_id']}:{env_vars['client_secret']}"
    creds_b64 = base64.b64encode(creds.encode())
    headers= {"Authorization": f"Basic {creds_b64.decode()}"}
    data= {"grant_type": "client_credentials"}
    token = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
    token = token.json()['access_token']
    return {"Accept": "application/json", "Content-Type": "application/json", "Authorization": f"Bearer {token}"}

async def get_artist(session: ClientSession, artist_uri: str) -> dict:
    """
    Given an artist's URI, return their info.
    """
    uri = artist_uri.split(":")[-1]
    endpoint = f"https://api.spotify.com/v1/artists/{uri}"

    async with session.get(endpoint) as response:
        response = await(response.json())

    return response

In [15]:
with open("./data/artists/artist_uris.txt", "r") as artist_data:
    artist_uris = [uri.strip() for uri in artist_data.readlines()]
async with ClientSession(headers=get_header_with_token(env_vars['client_id'], env_vars)) as session:
    tasks = [asyncio.ensure_future(get_artist(session, uri)) for uri in artist_uris]
    artist_info = await(asyncio.gather(*tasks))
artist_info

ContentTypeError: 0, message='Attempt to decode JSON with unexpected mimetype: ', url=URL('https://api.spotify.com/v1/artists/6rVWskb8AOPPak8qg6dh4e')

In [ ]:
# import json
#
# with open("./artists/artist_popularity_data.txt", "w") as f:
#     json.dump(artist_info, f)